In [38]:
import numpy as np
from typing import Any, Callable, Iterable, Iterator, Optional, Union, Tuple
import utils

Arr = np.ndarray

In [39]:
def log_back(grad_out: Arr, out: Arr, x: Arr) -> Arr:
    '''Backwards function for f(x) = log(x)

    grad_out: gradient of some loss wrt out
    out: the output of np.log(x)
    x: the input of np.log

    Return: gradient of the given loss wrt x
    '''
    return grad_out * (1 / x)

utils.test_log_back(log_back)

All tests in `test_einsum_inner` passed!


In [40]:
def unbroadcast(broadcasted: Arr, original: Arr) -> Arr:
    '''Sum 'broadcasted' until it has the shape of 'original'.

    broadcasted: An array that was formerly of the same shape of 'original'
    and was expanded by broadcasting rules.
    '''
    # Sum and remove dimensions that were prepended to the front of the original shape.
    n_dims_prepended = len(broadcasted.shape) - len(original.shape)
    unbroadcasted = broadcasted.sum(axis=tuple(range(n_dims_prepended)))

    # Sum dimensions that were originally 1 back to the size 1 (using keepdims=True).
    for dim, os in enumerate(original.shape):
        if os == 1:
            unbroadcasted = unbroadcasted.sum(axis=dim, keepdims=True)
    
    return unbroadcasted

utils.test_unbroadcast(unbroadcast)

All tests in `test_unbroadcast` passed!


In [41]:
def multiply_back0(grad_out: Arr, out: Arr, x: Arr, y: Union[Arr, float]) -> Arr:
    """Backwards function for x * y wrt argument 0 aka x."""
    if not isinstance(y, Arr):
        y = np.array(y)
    return unbroadcast(y * grad_out, x)


def multiply_back1(grad_out: Arr, out: Arr, x: Union[Arr, float], y: Arr) -> Arr:
    """Backwards function for x * y wrt argument 1 aka y."""
    if not isinstance(x, Arr):
        x = np.array(x)
    return unbroadcast(x * grad_out, y)


utils.test_multiply_back(multiply_back0, multiply_back1)
utils.test_multiply_back_float(multiply_back0, multiply_back1)

All tests in `test_multiply_back` passed!
All tests in `test_multiply_back_float` passed!


In [42]:
def forward_and_back(a: Arr, b: Arr, c: Arr) -> Tuple[Arr, Arr, Arr]:
    '''
    Calculates the output of the computational graph above (g), then backpropogates the gradients and returns dg/da, dg/db, and dg/dc
    '''
    d = a * b
    e = np.log(c)
    f = d * e
    g = np.log(f)

    final_grad_out = np.array([1.0])

    dg_df = log_back(grad_out=final_grad_out, out=g, x=f)
    dg_dd = multiply_back0(dg_df, f, d, e)
    dg_de = multiply_back1(dg_df, f, d, e)

    dg_da = multiply_back0(dg_dd, d, a, b)
    dg_db = multiply_back1(dg_dd, d, a, b)
    dg_dc = log_back(dg_de, e, c)

    return dg_da, dg_db, dg_dc


utils.test_forward_and_back(forward_and_back)

All tests in `test_forward_and_back` passed!
